In [1]:
import os
import tkinter as tk
from tkinter import filedialog
import pandas as pd
import matplotlib.pyplot as plt
import time
from tqdm import tqdm
import re
from datetime import datetime, timedelta, date
# file_name = '2021_09_01_23_50_00.csv' # Contains time that spans over midnight
# file_name = '2021_09_08_15_50_37.csv' # Contains PLC_TIME and PLC_TIME_CV
# file_name = '2021_09_01_16_08_41.csv' # Does not contain PLC_TIME or PLC_TIME_CV
file_name = str()
start_date = datetime
df = pd.DataFrame()

In [2]:
def set_timon_log_filename():
    global file_name
    root = tk.Tk()
    root.withdraw()
    print('Select a TiMon Logfile: ')
    file_name = filedialog.askopenfilename(title='Select TiMon log file')
    root.destroy()

In [3]:
def read_and_clean_timon_log():
    global file_name
    df = pd.read_csv(file_name, sep=';')
    df.replace(',','.', regex=True, inplace=True)
    df['TIME'] = df['TIME'].astype('float64')
    convert_bitsets_to_int()
    return df

In [4]:
def set_start_date_var():
    global start_date
    date_list = [int(i) for i in file_name.split('.')[0].split('_')]
    start_date = datetime(date_list[0], date_list[1], date_list[2], date_list[3], date_list[4], date_list[5])
    print(f'Set start_date var to: {start_date}')

In [5]:
def create_datetime_column():
    global start_date
    global df
    if 'PLC_TIME(Timedate48)' in df.columns and 'PLC_TIME_CV(Enum2)' in df.columns:
        df['Time Date'] = pd.to_datetime(df['PLC_TIME(Timedate48)'], unit='s') + pd.to_timedelta(df['PLC_TIME_CV(Enum2)'], unit='h')
        df = df.set_index(['Time Date'])
    else:
        set_start_date_var()
        df['Time Date'] = df['TIME'].apply(lambda x: start_date + timedelta(seconds=x))
        df = df.set_index(['Time Date'])
    return df

In [6]:
def create_excel_format():
    global file_name
    global df
    root = tk.Tk()
    root.withdraw()
    df.to_csv(filedialog.asksaveasfilename(defaultextension='.csv', 
                                 title='EXCEL_Timon', filetypes=[('csv files', '*.csv')], initialfile=f'EXCEL_{file_name}'))
    root.destroy()

In [7]:
set_timon_log_filename()
df = read_and_clean_timon_log()
create_datetime_column()

Select a TiMon Logfile: 


,TIME,PLC_TIME(Timedate48),PLC_TIME_CV(Enum2),FDS_LIFEWORD(Unsigned8),GWE_MVB_ETH_CAR10ST(Enum2),GWE_MVB_ETH_CAR11ST(Enum2),GWE_MVB_ETH_CAR12ST(Enum2),GWE_MVB_ETH_CAR13ST(Enum2),GWE_MVB_ETH_CAR14ST(Enum2),GWE_MVB_ETH_CAR15ST(Enum2),...,PLC_LeadingDir(Unsigned8),PLC_LeadingReq(Unsigned8),PLC_MASTER_COACH(Unsigned16),PLC_EVR_BS1(Bitset8),ASDO_StsW(Bitset16),ASDO_NumCoaches(Unsigned8),DO1A_LifeB(Unsigned8),DO1B_LifeB(Unsigned8),DO2A_LifeB(Unsigned8),DO2B_LifeB(Unsigned8)
Time Date,,,,,,,,,,,,,,,,,,,,,
2021-09-08 15:50:39,0.000,1631112639.000,1,247,3,3,3,3,3,3,...,1,2,15005,8c,c000,0,50,51,51,53
2021-09-08 15:50:39,0.512,1631112639.000,1,249,3,3,3,3,3,3,...,1,2,15005,8c,c000,0,54,56,55,58
2021-09-08 15:50:40,1.024,1631112640.000,1,249,3,3,3,3,3,3,...,1,2,15005,8c,c000,0,59,61,60,63
2021-09-08 15:50:40,1.536,1631112640.000,1,251,3,3,3,3,3,3,...,1,2,15005,8c,c000,0,64,65,64,67
2021-09-08 15:50:41,2.048,1631112641.000,1,251,3,3,3,3,3,3,...,1,2,15005,8c,c000,0,68,70,69,72
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-09-08 15:55:24,284.672,1631112924.000,1,52,3,3,3,3,3,3,...,1,2,15005,8c,c000,0,78,79,78,81
2021-09-08 15:55:24,285.184,1631112924.000,1,54,3,3,3,3,3,3,...,1,2,15005,8c,c000,0,82,84,83,86
2021-09-08 15:55:25,285.696,1631112925.000,1,54,3,3,3,3,3,3,...,1,2,15005,8c,c000,0,87,89,88,90


In [14]:
def convert_bitsets_to_int():
    global df
    for col in df.columns:
        if 'Bitset' in col and df[col].dtypes == 'object':
            df[col] = df[col].apply(int, base=16)

In [15]:
df['PLC_A_HEARBEAT(Bitset16)']

Time Date
2021-09-08 15:50:39    45702
2021-09-08 15:50:39    47750
2021-09-08 15:50:40    49798
2021-09-08 15:50:40    51846
2021-09-08 15:50:41    53894
                       ...  
2021-09-08 15:55:24     4742
2021-09-08 15:55:24     6790
2021-09-08 15:55:25     8838
2021-09-08 15:55:25    10886
2021-09-08 15:55:26    12934
Name: PLC_A_HEARBEAT(Bitset16), Length: 561, dtype: int64